In [12]:
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found") 

import re
import sqlite3

In [13]:
conn = sqlite3.connect('referencias_cat.sqlite')
cur = conn.cursor()

In [14]:
cur.executescript('''
CREATE TABLE IF NOT EXISTS referencias ( 
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, 
    referencia TEXT UNIQUE
    );
CREATE TABLE IF NOT EXISTS titulos ( 
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, 
    titulo TEXT,
    referencia_id INTEGER
    );
CREATE TABLE IF NOT EXISTS urls ( 
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    url TEXT, 
    titulo_id INTEGER 
    );
    ''')

In [15]:
fhand = open('Referencias.txt','r',encoding = 'utf-8')
for line in fhand:
    line = line.strip()
    cur.execute('''INSERT OR IGNORE INTO referencias (referencia) VALUES ( ? )''', ( line, ) )
    cur.execute('SELECT id from referencias WHERE referencia = ?', ( line,))
    referencia_id = cur.fetchone()[0]
    title = re.findall('“.+”', line)
    if title == []: 
                title = [line,]
    cur.execute('INSERT OR IGNORE INTO titulos ( titulo, referencia_id ) VALUES ( ?, ? )', ( title[0], referencia_id ) )
    cur.execute('SELECT id from titulos WHERE  titulo = ?', ( title[0], ))
    titulo_id = cur.fetchone()[0]
    for url in search(title[0], tld="com", num=1, stop=1, pause=2):
                cur.execute('INSERT OR IGNORE INTO urls ( url, titulo_id ) VALUES( ?, ? )', ( url, titulo_id))
conn.commit()

In [16]:
refhand = cur.execute('''
    SELECT referencia, titulo, url 
    from urls JOIN titulos JOIN referencias 
    on referencias.id = titulos.referencia_id AND titulos.id = urls.titulo_id
    ''')
for referencia, titulo, url in refhand:
    print(referencia)
    print(titulo)
    print(url + '\n')

Adalberto, P R, A C Massabni, E C Carmona, A J Goulart, D. P. Marques, and R. Monti. 2010. “Effect of Divalent Metal Ions on the Activity and Stability of β-Galactosidase Isolated from Kluyveromyces Lactis.” Journal of Basic and Applied Pharmaceutical Sciences 31 (3): 143–150.
“Effect of Divalent Metal Ions on the Activity and Stability of β-Galactosidase Isolated from Kluyveromyces Lactis.”
https://www.researchgate.net/publication/49599600_Effect_of_divalent_metal_ions_on_the_activity_and_stability_of_b-galactosidase_isolated_from_Kluyveromyces_lactis

Albuquerque, T. L., Pierce, S, Rueda, M., Marzocchella A, Luciana R.B. Gonçalves, M.V.P.R.P, Lafuente,R.F. 2016. “Ion Exchange of β-Galactosidase: The Effect of the Immobilization PH on Enzyme Stability.” Process Biochemistry 51 (7): 875–880. doi:10.1016/j.procbio.2016.03.014.
“Ion Exchange of β-Galactosidase: The Effect of the Immobilization PH on Enzyme Stability.”
https://www.sciencedirect.com/science/article/pii/S1359511316300447

A

In [17]:
conn.close()   